# All the Imports

In [1]:
import sys
import os
import logging
sys.path.insert(1, '../src/')

In [2]:
%%time 
import itertools 
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
from lassonet import LassoNetClassifier


from RankCorr.picturedRocks import Rocks
import smashpy as smashpy


import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

from utils import *


CPU times: user 8.03 s, sys: 705 ms, total: 8.73 s
Wall time: 8.15 s


In [3]:
from os import path

In [4]:
model_save_path = '../data/{}/models/'
num_times = 10

  

In [5]:
data_locations = [
                  'CITE-seq',
                 # 'developmental_mouse',
                  'mouse_brain_broad',
                  'paul',
                  'zeisel'
                 # 'synthetic'
]

In [6]:
result_axis = [
 ('All Markers', 'all_markers_results_{}.npy', 'all_markers_knn_results_{}.npy'), 
 ('L1 VAE', 'l1_vae_results_{}.npy', 'l1_vae_knn_results_{}.npy'),
 ('Smash RF', 'smash_rf_results_{}.npy', 'smash_rf_knn_results_{}.npy'),
 ('SMaSH DNN', 'smash_results_{}.npy', 'smash_knn_results_{}.npy'),
 ('RankCorr', 'rankcorr_results_{}.npy', 'rankcorr_knn_results_{}.npy'),
 ('Global Gate VAE', 'globalgate_vae_results_{}.npy', 'globalgate_vae_knn_results_{}.npy'),
 ('MarkerMap Unsupervised', 'marker_map_unsupervised_results_{}.npy', 'marker_map_unsupervised_knn_results_{}.npy'),
 ('MarkerMap Supervised', 'marker_map_supervised_results_{}.npy', 'marker_map_supervised_knn_results_{}.npy'),
 ('MarkerMap Mixed Supervision', 'marker_map_mixed_results_{}.npy', 'marker_map_mixed_knn_results_{}.npy'),
 ('Concrete VAE',  'concrete_vae_results_{}.npy', 'concrete_vae_knn_results_{}.npy'),
 ('LassoNet', 'lasso_net_results_{}.npy', 'lasso_net_knn_results_{}.npy')
]

## Mean Accuracies and F1

### Random Forest

In [7]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[1], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results1 = pd.DataFrame(results)
tabulated_results1

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.076, 0.901)","(0.132, 0.83)","(0.064, 0.919)","(0.067, 0.917)","(0.072, 0.909)","(0.132, 0.83)","(0.091, 0.885)","(0.063, 0.924)","(0.065, 0.92)","(0.127, 0.836)","(0.062, 0.924)"
mouse_brain_broad,"(0.009, 0.991)","(0.163, 0.827)","(0.018, 0.982)","(0.015, 0.984)","(0.032, 0.968)","(0.089, 0.906)","(0.013, 0.987)","(0.006, 0.994)","(0.031, 0.969)","(0.126, 0.865)","(0.008, 0.991)"
paul,"(0.184, 0.796)","(0.531, 0.406)","(0.114, 0.883)","(0.17, 0.817)","(0.149, 0.845)","(0.382, 0.573)","(0.139, 0.856)","(0.108, 0.889)","(0.138, 0.857)","(0.398, 0.555)","(0.126, 0.868)"
zeisel,"(0.042, 0.957)","(0.192, 0.794)","(0.049, 0.951)","(0.086, 0.911)","(0.045, 0.954)","(0.187, 0.801)","(0.124, 0.868)","(0.045, 0.955)","(0.066, 0.931)","(0.2, 0.785)","(0.054, 0.945)"


### KNN

In [8]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results2 = pd.DataFrame(results)
tabulated_results2

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.098, 0.898)","(0.191, 0.782)","(0.066, 0.922)","(0.067, 0.923)","(0.078, 0.907)","(0.18, 0.796)","(0.104, 0.885)","(0.062, 0.929)","(0.07, 0.92)","(0.18, 0.795)","(0.064, 0.926)"
mouse_brain_broad,"(0.103, 0.885)","(0.234, 0.744)","(0.025, 0.974)","(0.016, 0.983)","(0.041, 0.959)","(0.14, 0.848)","(0.015, 0.985)","(0.006, 0.994)","(0.041, 0.958)","(0.198, 0.782)","(0.009, 0.991)"
paul,"(0.321, 0.608)","(0.632, 0.375)","(0.217, 0.768)","(0.291, 0.676)","(0.257, 0.729)","(0.447, 0.521)","(0.268, 0.721)","(0.203, 0.789)","(0.253, 0.735)","(0.461, 0.513)","(0.228, 0.761)"
zeisel,"(0.054, 0.945)","(0.261, 0.726)","(0.047, 0.953)","(0.103, 0.892)","(0.047, 0.953)","(0.273, 0.717)","(0.153, 0.84)","(0.038, 0.961)","(0.067, 0.93)","(0.286, 0.7)","(0.059, 0.938)"


## Variances

### Random Forest

In [9]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    _, _, m, f1 = model_variances(model_save_path.format(e) + axis[1], num_times)
    m = round(m, 5)
    f1 = round(f1, 5)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results1 = pd.DataFrame(results)
tabulated_results1

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.00687, 0.01014)","(0.02216, 0.02942)","(0.00321, 0.00556)","(0.00688, 0.01011)","(0.00703, 0.00992)","(0.02341, 0.02825)","(0.01366, 0.01846)","(0.00575, 0.00769)","(0.00461, 0.00651)","(0.01707, 0.02263)","(0.00775, 0.01074)"
mouse_brain_broad,"(0.00118, 0.00125)","(0.03373, 0.03731)","(0.00264, 0.00262)","(0.00215, 0.00217)","(0.00319, 0.0032)","(0.03134, 0.03361)","(0.004, 0.00419)","(0.00065, 0.00069)","(0.01458, 0.01496)","(0.02833, 0.03041)","(0.00103, 0.00112)"
paul,"(0.01397, 0.01625)","(0.03289, 0.03068)","(0.0173, 0.01711)","(0.02674, 0.03112)","(0.01125, 0.01261)","(0.02965, 0.03472)","(0.01457, 0.01584)","(0.01222, 0.01232)","(0.0213, 0.02101)","(0.02604, 0.03466)","(0.00899, 0.01009)"
zeisel,"(0.00699, 0.00721)","(0.03541, 0.04028)","(0.00556, 0.00584)","(0.02172, 0.02375)","(0.00514, 0.00523)","(0.04568, 0.04929)","(0.0481, 0.05402)","(0.00626, 0.00648)","(0.01078, 0.01219)","(0.04423, 0.05143)","(0.00955, 0.00954)"


### KNN

In [10]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    _, _, m, f1 = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m, 5)
    f1 = round(f1, 5)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results2 = pd.DataFrame(results)
tabulated_results2

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.0054, 0.00517)","(0.03548, 0.04172)","(0.00445, 0.00537)","(0.00613, 0.00717)","(0.00629, 0.00834)","(0.02845, 0.03169)","(0.01622, 0.01721)","(0.00418, 0.00502)","(0.00494, 0.00637)","(0.0236, 0.02818)","(0.00527, 0.00735)"
mouse_brain_broad,"(0.00645, 0.00608)","(0.04626, 0.05147)","(0.00465, 0.00472)","(0.00345, 0.00345)","(0.00333, 0.00334)","(0.05128, 0.0591)","(0.00714, 0.00741)","(0.00077, 0.00077)","(0.02123, 0.02166)","(0.03769, 0.04341)","(0.00127, 0.00132)"
paul,"(0.01681, 0.02255)","(0.05892, 0.05146)","(0.01595, 0.01912)","(0.01848, 0.02353)","(0.0302, 0.03689)","(0.02446, 0.02886)","(0.01475, 0.01377)","(0.01735, 0.01849)","(0.0317, 0.03315)","(0.03431, 0.03568)","(0.02216, 0.02482)"
zeisel,"(0.00826, 0.00849)","(0.03513, 0.03642)","(0.00499, 0.00512)","(0.04191, 0.0458)","(0.00636, 0.00636)","(0.0561, 0.06126)","(0.04964, 0.05203)","(0.00455, 0.00465)","(0.01198, 0.01263)","(0.04426, 0.04958)","(0.01007, 0.01166)"


In [17]:
indices =  np.load('../data/paul/models/experiment_data_folds/rankcorr_indices_7.npy', allow_pickle = True)

In [18]:
indices2 =  np.load('../data/paul/models/experiment_data_folds/marker_map_mixed_indices_7.npy', allow_pickle = True)

In [19]:
indices[1]

array([1751, 2032,  173, 1158, 1663,  533, 1729,  885, 1905, 2699,  416,
       1839, 2566, 1571, 1097, 1154, 1047, 1005, 1754,  627, 1876, 1741,
       1730, 1561, 2612, 2479,  624, 2163, 1931, 2440, 1982, 2480,  558,
        855, 2188, 1622, 1245,  595,  439, 1844,   79,  930, 2587,  677,
         23, 1643,  579,  198,  577,  428, 1667, 1819, 2017, 1393, 2079,
       1020,  180,  459,  905,  258, 1765,  137, 1888, 2598,  255, 2646,
        316, 1426, 1786,  494,  168,  409, 1866, 1478,  141, 2097, 1830,
        860, 1572, 1129, 1959,  451,  985, 1030,  727, 1809, 1467, 1556,
       1411,  181,  593,  567,  656, 1666, 1384, 1938, 1070,  804, 2293,
        842, 1543,  103, 1328, 2285, 2513, 1604, 2243, 1108,  776, 1305,
        993, 1173,  474,  851, 2642, 1948,   69,  583, 2178,  136, 1976,
       2469, 1937, 2115,  499, 1167,  726,  426, 1671, 1325, 2013,  616,
        862,  476, 1246,  587,  867,   97, 1763, 1816,  105,  538,  442,
       2388,  259,  981, 2537, 1947,  223, 1335,  4

In [20]:
indices2[1]

array([1325,  300,  672, 1246,  655,  898, 1642, 1785,  897, 2595, 2382,
       1765, 2346, 2569,  781, 2135, 2296, 2116,  501, 1390, 2635, 2124,
       1180,  531,  837, 2099, 2183, 1840,  111,  259, 2312, 1055, 1371,
       2496, 1346, 2474,  963, 1712, 1677, 1123,  358, 2188, 1175, 1156,
       2157,  529, 1574, 2132, 2133,  362, 2139,  900, 2441, 1357, 1529,
       2503, 2162, 1413, 2628, 1613, 1894,  991,  123, 1002, 1295, 2365,
         64, 2253,   26, 2028, 1779, 1986, 2472, 2487,   97, 1703, 2502,
        727, 2332,  823, 2061, 1286, 2165,  249,  908, 2059, 2449, 1145,
        406,  198, 2233,  675, 2646, 1696, 1367, 1756, 1396, 2395, 1086,
       1947, 2004,  260, 1053, 2348, 2358, 2338, 1098, 2678, 1914, 2543,
       1663, 1479,  643, 2531, 2010, 1252,  859, 1219,  120, 1023,  234,
        839,  515,  600, 1849, 2171,  977,  296, 1487, 1929,  890,  100,
        185,  811,  647, 1401, 1630,  961, 2391, 1775,  470,  207,  755,
       2653,  812, 1339, 2584, 2202,  293,   90,  4